# Recomendador basado en contenidos
Consiste en recomendar contenido similar a un determinado contenido

#### Principales caracteristicas
- Sólo necesita la información de un producto, por lo que no se necesita información histórica
- Se suele usar para evitar problemas de arranque en frio

#### Como lo hacemos
1. Definir una métrica de similitud para comparar los contenidos
2. Calcular la similitud entre todos los elementos con la métrica seleccionada
3. (Opcional) Definir vecindarios para optimizar


## Antes de comenzar... veamos el dataset
Recursos turisticos de Andalucia obtenidos de [TripAdvisor](https://www.tripadvisor.es)

In [2]:
import pandas as pd

recursos = pd.read_csv('./datasets/Recursos_turisticos.csv', sep='|')
recursos.head(5)

,_id,nombre_ws,strCategoria,strSubcategoria,strCiudad,strDescripcion
0,5880b148636e564ad8d9b952,Hotel Albucasis,Alojamientos,Hoteles,Córdoba,.El hotel tiene a disposición de sus clientes ...
1,5880b148636e564ad8d9b963,Hotel Sierra Hidalga,Alojamientos,Hoteles,Ronda,Enclavado en una situación privilegiada con fa...
2,5880b1f2636e564ad8d9b969,Hotel Virgen de los Reyes,Alojamientos,Hoteles,Sevilla,"El Hotel Virgen de los Reyes***, ubicado en el..."
3,5880b1f2636e564ad8d9b96a,Hotel An Santo Domingo,Alojamientos,Hoteles,Lucena,El Hotel Santo Domingo Lucena está situado en ...
4,5880b1f2636e564ad8d9b96f,Leflet Valme,Alojamientos,Hoteles,Dos Hermanas,El Hotel Leflet Valme está situado estratégica...


## Metrica de similitud para texto

Una de las técnicas más utilizadas para obtener similitude texto es TF-IDF, tecnica proveniente del campo de Informatin Retrieval.

TF-IDF permite realizar una representación vectorial de los textos. Por ejemplo:

> "En un lugar de la mancha"

|_|En|un |lugar|de|la|mancha|
|--|--|---|-----|--|--|------|
|TF |1|1|1|1|1|1|
|DF |200|1700|100|3000|4075|3|
|IDF|1/200|1/1700|1/100|1/3000|1/4075|1/3|

Al tener una representación vectorial de los textos podemos usar medidas tradicionale
<img src="./imagenes/similitudes.png">

### Ejemplo TF-IDF

**Paso 1** Preparacion y tokenizado de datos

In [11]:
import gensim
import nltk
import tempfile
import heapq

from nltk.tokenize import word_tokenize
nltk.download('punkt')

documentos = recursos['strDescripcion']
print ("Numero de documentos:", len(documentos))
print (documentos[1])

docs_tokenizado = [[w.lower() for w in word_tokenize(text)] 
            for text in documentos]

print ("\n\nElementos tokenizados\n")
print (docs_tokenizado[1])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Numero de documentos: 601
Enclavado en una situación privilegiada con fantásticas vistas a la sierra, y a menos de 2 Km. del centro de la ciudad de Ronda y 45 Km. de la Costa del Sol, es un lugar ideal para disfrutar de la oferta cultural y gastronómica además del patrimonio histórico-artístico de la ciudad, pudiendo combinarla con actividades en su entorno natural como la Sierra de las Nieves (reserva de la biosfera). Su decoración de estilo moderno y actual ofrece un ambiente selecto y acogedor. En nuestro restaurante podrán degustar los platos de nuestro cocinero Antonio Castro Aguilar además de una exquisita carta de vinos..


Elementos tokenizados

['enclavado', 'en', 'una', 'situación', 'privilegiada', 'con', 'fantásticas', 'vistas', 'a', 'la', 'sierra', ',', 'y', 'a', 'menos', 'de', '2', 'km', '.', 'del', 'centro', 'de', 'la', 'ciudad', 'de', 'ronda', 'y', '45', 'km', '.', 'de', 'la', 'costa', 'del', 'sol', ',', 'es', 'un', 'lugar', 'ideal', 'para', 'disfrutar', 'de', 'la', 'ofe

**Paso 2** Creación de un un diccionario con todo el vocabulario

In [12]:
diccionario = gensim.corpora.Dictionary(docs_tokenizado)

print("Número de palabras en el diccionario:",len(diccionario))
for i in range(len(diccionario)):
    print(i, diccionario[i])

Número de palabras en el diccionario: 7932
0 ,
1 .
2 .el
3 100
4 3
5 30
6 a
7 albolafia
8 amantes
9 antiguo
10 aves
11 barrio
12 belleza
13 bus
14 calle
15 cama
16 ciudad
17 clientes
18 coche
19 cual
20 cuna
21 córdoba
22 de
23 del
24 describir
25 diferentes
26 disposición
27 donde
28 en
29 encuentra
30 especies
31 esta
32 guadalquivir
33 hermosa
34 hotel
35 información
36 judería
37 la
38 los
39 mezquita
40 minutos
41 monumentos
42 morena
43 muchos
44 museos
45 más
46 naturaleza
47 no
48 nuestra
49 o
50 observar
51 oficina
52 para
53 pelo
54 pie
55 poco
56 posee
57 puede
58 pueden
59 que
60 recomendamos
61 relajarnos
62 restaurantes
63 ribera
64 rincón
65 rio
66 se
67 secador
68 sierra
69 situados
70 sotos
71 supletoria
72 sus
73 tabernas
74 tanta
75 televisión
76 tiene
77 todo
78 tranquilo
79 turismo
80 turística.estamos
81 típica
82 un
83 una
84 visitar
85 y
86 (
87 )
88 ..
89 2
90 45
91 acogedor
92 actividades
93 actual
94 además
95 aguilar
96 ambiente
97 antonio
98 biosfera
99 car

1401 silencio
1402 tener
1403 tomando
1404 tomar
1405 tradicionales
1406 vas
1407 vino
1408 barrameda
1409 bellas
1410 casa-palacio
1411 conjunto
1412 edificios
1413 maravillosas
1414 nobles
1415 repleta
1416 sanlúcar
1417 40
1418 castillos
1419 encantador
1420 fortalezas
1421 fuenmayor
1422 impresionantes
1423 infantil
1424 mágina
1425 nacimientos
1426 personal
1427 proporcionar
1428 puerto
1429 tumbonas
1430 valle
1431 visita
1432 x
1433 azotea
1434 cerrada
1435 contactar
1436 fechas
1437 gris
1438 mantenimiento
1439 permanecerá
1440 privados
1441 puntuales
1442 público
1443 trabajos
1444 50
1445 52
1446 ?
1447 andaluz
1448 ascensor
1449 año
1450 bicicletas
1451 bordes
1452 conocer
1453 cruzan
1454 céntrica
1455 dando
1456 encaraman
1457 espectaculares
1458 exclusivo
1459 garganta
1460 gratis
1461 infinidad
1462 lejos
1463 llamar
1464 muebles
1465 naturales
1466 necesita
1467 ocho
1468 paisaje
1469 parques
1470 peligrosamente
1471 precio
1472 próxima
1473 puentes
1474 qué
1475 realme

2532 estacias
2533 gustoso
2534 hacerlas
2535 incomparable
2536 personaliza
2537 provistas
2538 proximidad
2539 renunciar
2540 rústica
2541 sociales
2542 unen
2543 unidos
2544 visitante
2545 íntimo
2546 únicas
2547 79
2548 apenas
2549 armarios
2550 cauchiles-
2551 cervino-
2552 club-
2553 contar
2554 cubierto
2555 descubierto
2556 esquiando
2557 ghm
2558 guardaesquies-
2559 health
2560 horas-
2561 hotel.en
2562 i
2563 iniciados
2564 kílometros
2565 material
2566 mediante
2567 monachil
2568 montblanc-
2569 parador
2570 personas.todas
2571 recepcion
2572 salir
2573 salones-atrium-
2574 snowboard.todos
2575 sulayr-
2576 telesilla
2577 terraza-snack
2578 va
2579 volver
2580 559
2581 casera
2582 plácidamente
2583 viaje
2584 28
2585 35
2586 apostado
2587 apreciado
2588 aquellos
2589 bonitos
2590 busquen
2591 diversas
2592 dúrcal
2593 emociones
2594 especialidad
2595 freiduría
2596 fuertes
2597 gerencia
2598 haciendo
2599 idea
2600 iglesias
2601 intento
2602 invitando
2603 lata
2604 lecrin
26

4050 divisa
4051 fundamental
4052 invade
4053 pieza
4054 responde
4055 situadas
4056 sobrevolando
4057 vanguardistas
4058 vidrio
4059 colonial
4060 cristina
4061 demanda
4062 grand
4063 nace
4064 perder
4065 convierta
4066 puertatierra
4067 140
4068 comer
4069 exitosa
4070 fresco
4071 satisfarán
4072 ventanales
4073 66
4074 bares.el
4075 benalmádena
4076 carihuela
4077 descubierta
4078 garantizarle
4079 hacerle
4080 nocturna
4081 paquito
4082 periodo
4083 pubs
4084 2.5
4085 tryp
4086 750
4087 alojado
4088 ancho
4089 animales
4090 cañones
4091 cubo
4092 difusores
4093 diurnas
4094 familia¡
4095 galeón
4096 giro
4097 irregular
4098 lámina
4099 mástiles
4100 niño
4101 niños¡
4102 nocturnas
4103 park
4104 patos
4105 pirata
4106 piscina/playa
4107 rectangular
4108 rectos
4109 rocas
4110 serpiente
4111 seta
4112 splash
4113 superficie
4114 tematizada
4115 tematizado
4116 temática
4117 tiburón
4118 toalla
4119 tubo
4120 velas
4121 ¡diversión
4122 ¡especial
4123 75
4124 ac
4125 arqueológico
41

5581 abentofail
5582 adquieran
5583 allá
5584 asentamientos
5585 autodidacta
5586 autor
5587 definida
5588 exclusivamente
5589 faltar
5590 favor
5591 filósofo
5592 frecuentes
5593 gata-níjar
5594 ibn
5595 indonesia
5596 insuperable
5597 llegadas
5598 matemático
5599 médico
5600 novela
5601 obviamente
5602 recibe
5603 recicladas
5604 salidas
5605 traídas
5606 tufail
5607 utilizado
5608 vecinas
5609 –níjar
5610 califa
5611 culto
5612 perfil
5613 alcornocales
5614 ceuta
5615 estrategica
5616 estrecho
5617 guiada
5618 hipercor
5619 tetuan
5620 turistica
5621 vecina
5622 visitor
5623 aquapolis
5624 equipados.el
5625 playa.el
5626 temporada.los
5627 bib-rambla
5628 conservado
5629 exponente
5630 mesones
5631 píe
5632 colorista
5633 convencional
5634 emotions
5635 fortuny
5636 imagine
5637 planteadas
5638 vives
5639 indalo
5640 mojácar
5641 despacio
5642 integra
5643 interiorismo
5644 alquilar
5645 atarazanas
5646 carga
5647 descarga
5648 facilitar
5649 genial
5650 gluten
5651 interesados
565

7091 folletos
7092 guiadas…
7093 jentoft
7094 limpiamos
7095 oro…
7096 vigilado
7097 alquilan
7098 bar-restaurante
7099 escaleras
7100 facilidad
7101 roca
7102 épocas
7103 164
7104 alojarte
7105 delicadeza
7106 pensar
7107 sobrias
7108 tengan
7109 258
7110 elíptica
7111 encierra
7112 encuentros
7113 estadios
7114 facilita
7115 adiafa
7116 barbate
7117 breña
7118 conjugando
7119 diáfanos
7120 numerosa
7121 permitiéndole
7122 serenas
7123 servicial
7124 sorprendentes
7125 vejer
7126 192
7127 bar-
7128 bomba
7129 cibercafé-restaurante
7130 garrucha
7131 l
7132 lavavajillas
7133 levante
7134 menaje
7135 neurálgico
7136 playazo
7137 quiosco
7138 700m
7139 asfáltica
7140 colorao
7141 federación
7142 naútica
7143 1850
7144 artesonado
7145 ciudad.nuestras
7146 imaginativa
7147 inteligente
7148 manierista
7149 neomudéjar
7150 preside
7151 representativo
7152 servirán
7153 ustedes
7154 4x4
7155 berlinas
7156 furgonetas
7157 lucía
7158 50m
7159 playalinda
7160 barcos
7161 cuervo
7162 plana.el
716

**Paso 3** Generación del corpus de documentos, se calcula el número de veces que cada palabra aparece en el documento (TF)

In [13]:
corpus = [diccionario.doc2bow(gen_doc) for gen_doc in docs_tokenizado]
print(corpus[67])

[(1, 1), (22, 1), (23, 3), (28, 1), (34, 2), (59, 1), (85, 2), (101, 1), (105, 1), (136, 1), (160, 3), (209, 1), (212, 1), (242, 1), (351, 1), (377, 1), (676, 1), (791, 1), (1380, 1), (2259, 1), (2260, 1), (2261, 1), (2262, 1), (2263, 1), (2264, 1)]


**Paso 4** Generación del modelo para TF-IDF

In [14]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)
tf_idf[corpus[45]]

TfidfModel(num_docs=601, num_nnz=54911)
54911


[(0, 0.00814028941284755),
 (1, 0.004510845802886568),
 (5, 0.08457220354266493),
 (6, 0.030003152041329653),
 (16, 0.03504664991688565),
 (22, 0.0024849717258417334),
 (23, 0.007695324425977478),
 (28, 0.007138887139983436),
 (34, 0.006440077474554821),
 (37, 0.02585862957028758),
 (52, 0.07094413463188166),
 (68, 0.11777534161572792),
 (77, 0.03863059963504467),
 (84, 0.07776532503135802),
 (85, 0.00709553876349057),
 (86, 0.038091401694130174),
 (87, 0.038091401694130174),
 (104, 0.059247012824668856),
 (105, 0.01178409754994704),
 (119, 0.10378809703534718),
 (125, 0.05924821993213235),
 (136, 0.08296747553351903),
 (140, 0.05615057431929756),
 (159, 0.041885702513057545),
 (160, 0.014063299341742216),
 (198, 0.11497530117867365),
 (234, 0.04970995297707485),
 (239, 0.0661589203239136),
 (242, 0.04193220110111866),
 (244, 0.1249695031323539),
 (252, 0.058531439741026044),
 (254, 0.07167372781027485),
 (257, 0.056479251017738416),
 (262, 0.04970995297707485),
 (267, 0.06150841606642

**Paso 5** Creación de un índice temporal

In [17]:
print (tempfile.gettempdir())
sims = gensim.similarities.Similarity(tempfile.gettempdir(),tf_idf[corpus],
                                      num_features=len(diccionario))
print (sims[tf_idf[corpus[45]]])

C:\Users\Mario\AppData\Local\Temp
[3.46628390e-02 4.94221449e-02 8.45900364e-03 1.63472015e-02
 6.03292920e-02 1.50345964e-02 4.00197655e-02 7.68848136e-02
 6.83562714e-04 3.66462767e-02 5.04660457e-02 1.45551100e-01
 1.18446192e-02 9.74777807e-03 2.57724877e-02 5.18233255e-02
 5.50403371e-02 9.58691165e-03 2.08421052e-02 4.61889373e-04
 4.34443951e-02 2.38070916e-02 5.17617576e-02 3.73220593e-02
 9.28834230e-02 9.17329565e-02 6.23419285e-02 7.57740289e-02
 3.72250983e-03 1.43007468e-02 4.98469025e-02 2.27099806e-02
 3.49135920e-02 3.79747897e-03 6.39672279e-02 4.21048515e-02
 4.41701151e-02 5.69540821e-02 4.52524200e-02 5.68748787e-02
 9.32489615e-03 2.54631136e-02 6.25247881e-02 5.41383959e-02
 2.02734042e-02 9.99999762e-01 3.60879079e-02 6.89504743e-02
 2.10267343e-02 2.03620084e-02 3.01039144e-02 3.79183330e-03
 4.91288900e-02 6.29052147e-02 3.58853228e-02 3.56538072e-02
 2.22602766e-02 2.11815909e-02 1.83550417e-02 3.72650251e-02
 1.24560501e-02 4.40983176e-02 2.75598075e-02 8.310

**Paso 6** Obtener elementos recomendados, los más similares

In [19]:
salida = sims[tf_idf[corpus[45]]]

heap = []
indice = 0
for medida in salida:
    heapq.heappush(heap, (medida, documentos[indice]))
    indice +=1

print (documentos[45])
heapq.nlargest(10,heap)

Situada en el pueblo de La Zubia, junto al Parque Natural de Sierra Nevada y tan solo 5 min de la capital nazarí. Ideal tanto para hacer excursiones y practicar senderismo como para visitar La Alhambra y la ciudad. Fácil acceso por autovía A- 92 10 min a La Alhambra, 20 min a la estación de esquí de Sierra Nevada, 25 a la Alpujarra y a 30 la playa. Chimenea para relajarse al calor de la lumbre, salón amplio, dividido en dos ambientes, con ventana de acceso a la cocina y tv grande (ideal para reuniones grandes). La cocina dispone de todo lo necesario, patio andaluz, 12 dormitorios con mucha luz. Disponemos de restaurante especializado en carnes a la brasa, tanto el hotel como el restaurante están rodeados de jardines para tomar el sol en cualquier época del año.


[(0.99999976,
  'Situada en el pueblo de La Zubia, junto al Parque Natural de Sierra Nevada y tan solo 5 min de la capital nazarí. Ideal tanto para hacer excursiones y practicar senderismo como para visitar La Alhambra y la ciudad. Fácil acceso por autovía A- 92 10 min a La Alhambra, 20 min a la estación de esquí de Sierra Nevada, 25 a la Alpujarra y a 30 la playa. Chimenea para relajarse al calor de la lumbre, salón amplio, dividido en dos ambientes, con ventana de acceso a la cocina y tv grande (ideal para reuniones grandes). La cocina dispone de todo lo necesario, patio andaluz, 12 dormitorios con mucha luz. Disponemos de restaurante especializado en carnes a la brasa, tanto el hotel como el restaurante están rodeados de jardines para tomar el sol en cualquier época del año.'),
 (0.17315687,
  'Nuestro hotel Alicia Carolina está situado en un lugar estratégico, tanto para visitar el centro histórico de Granada y la Alhambra (7 min en coche), como adentrarse en el maravilloso parque 

## Nuevas técnicas, deep learning

Es una nueva técnica dentro del aprendizaje automático, basado en la estructura de las redes neuronales, pero usando arquitecturas profundas (deep) en la que múltiples capas son conectadas.

La principal característica y ventaja del deep learning es el **feature discovering** sin la necesidad de la intervención humana.

### Paragraph vector

- Algoritmo de aprendizaje no supervisado capaz de generar una representacion vectorial a partir de texto.
- Paragraph vector es capaz de aprender la semántica de las palabras.

<img src="./imagenes/doc2vec.jpg">

#### Ejemplo

**Paso 1** Entrenamiento del modelo

In [24]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Configuración
#-------------
__longitud_vector = 5
__ventana = 10
__frecuencia_minima = 5
__learning_rate = 0.025
__min_learning_rate = 0.025
__epoch = 100
#---------------------

                
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(documentos)]
model = gensim.models.Doc2Vec(vector_size=__longitud_vector, window=__ventana, min_count=__frecuencia_minima, workers=11,alpha=__learning_rate, min_alpha=__min_learning_rate, epochs=__epoch)
model.build_vocab(tagged_data)
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

#Recorrido de todos los documentos con los que se ha entrenado
for i in range(20):
    print (model.dv[i].tolist())
    


[0.9831425547599792, 3.085474729537964, 0.4872993528842926, 2.3987786769866943, -2.4624197483062744]
[1.7566848993301392, 0.6382661461830139, 2.568601369857788, 0.8350446820259094, -1.4669500589370728]
[2.232905626296997, 1.5502232313156128, 0.8649148344993591, 2.970885753631592, 0.9132641553878784]
[1.7300124168395996, 3.4043073654174805, 3.167577028274536, 4.218224048614502, 1.7721965312957764]
[-1.2096055746078491, 1.3010120391845703, 2.975905418395996, 1.972477912902832, 0.8949034810066223]
[3.5742688179016113, -2.1101462841033936, 3.7760071754455566, 2.548618793487549, -1.2012343406677246]
[0.1421697437763214, 0.028751451522111893, 2.0401012897491455, -2.508566379547119, -1.059140920639038]
[2.1600310802459717, -2.469465970993042, 3.7264857292175293, 1.1203092336654663, 1.7073003053665161]
[-1.685178518295288, 2.4054043292999268, 1.677486777305603, 0.6898583173751831, 1.9127554893493652]
[3.989696979522705, 1.2374504804611206, 1.8956832885742188, 4.742537021636963, -1.609123945236

**Paso 2** Cálculo de similitud

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

elemento =model.dv[202]

heap = []
for i in range(len(model.dv)):
    sim =cosine_similarity(elemento.reshape(1, -1), model.dv[i].reshape(1, -1))
    heapq.heappush(heap, (sim, documentos[i]))

print (documentos[202])
heapq.nlargest(10,heap)

Vincci Selección Estrella del Mar 5* es un nuevo establecimiento cinco estrellas perteneciente a la cadena Vincci Hoteles.Situado en el municipio de Marbella, a 8 km del centro y a 43 km del aeropuerto de Málaga, el Vincci Selección Estrella del Mar se encuentra en una zona de excepción en pleno centro de la Costa del Sol, en la zona conocida como “Elviria–Las Chapas”, considerada una de las mejores zonas del municipio y que cuenta, sin lugar a dudas, con la mejor playa de esta parte de la costa.De moderno estilo mediterráneo con influencias árabes, cuenta con 137 habitaciones y suites impregnadas del estilo y elegancia que caracteriza a Vincci Hoteles. ﻿


[(array([[1.]], dtype=float32),
  'Vincci Selección Estrella del Mar 5* es un nuevo establecimiento cinco estrellas perteneciente a la cadena Vincci Hoteles.Situado en el municipio de Marbella, a 8 km del centro y a 43 km del aeropuerto de Málaga, el Vincci Selección Estrella del Mar se encuentra en una zona de excepción en pleno centro de la Costa del Sol, en la zona conocida como “Elviria–Las Chapas”, considerada una de las mejores zonas del municipio y que cuenta, sin lugar a dudas, con la mejor playa de esta parte de la costa.De moderno estilo mediterráneo con influencias árabes, cuenta con 137 habitaciones y suites impregnadas del estilo y elegancia que caracteriza a Vincci Hoteles. \ufeff'),
 (array([[0.99801457]], dtype=float32),
  'El Hotel Almerimar se encuentra en un enclave privilegiado de la Costa de Almeria, en primera línea de playa en la exclusiva urbanización Almerimar. El Hotel dispone de 278 habitaciones totalmente equipadas, con inmejorables vistas al mar o campo de 